In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

# 1. Veriyi Yükleme
# csv dosyasının kod ile aynı klasörde olduğundan emin ol
df = pd.read_csv('df_easy.csv')

# 2. Veri Temizliği ve Gruplandırma
# CSV'deki uzun cevapları kısa etiketlere (Teist, Ateist vb.) dönüştürüyoruz.
belief_map = {
    "Kişisel bir tanrı’nın var olduğuna inanıyorum.": "Teist",
    "Hiçbir tanrı veya ilahi gücün var olmadığına inanıyorum.": "Ateist",
    "Tanrı veya ilahi bir gücün var olup olmadığından emin değilim.": "Agnostik",
    "Evrenin veya tüm varoluşun kendisinin ilahi bir bütün olduğuna inanıyorum.": "Panteist",
    "Bir yaratıcı gücün var olduğuna inanıyorum ama bu gücün dünyaya müdahale etmediğini düşünüyorum.": "Deist",
    "Diğer": "Diğer"
}

# .str.strip() komutu, metinlerin başındaki/sonundaki görünmez boşlukları temizler (hatayı önler)
df['inanc_kategori'] = df['inanc_turu'].str.strip().map(belief_map).fillna('Diğer')

# İnceleyeceğimiz pratikler (Sütun isimlerinin CSV ile birebir aynı olması gerek)
practices = ['Dua', 'Kader', 'Ilahi_Adalet', 'Reenkarnasyon', 'Cekim_Yasasi', 'Ruh']

# 3. Yüzdelik Oranların Hesaplanması
# Her inanç grubu için bu pratikleri yapma ortalamasını alıp 100 ile çarpıyoruz
practice_percentages = df.groupby('inanc_kategori')[practices].mean() * 100

print("--- Yüzdelik Dağılım Tablosu ---")
print(practice_percentages)
print("\n" + "-"*30 + "\n")

# 4. İstatistiksel Test: Ki-Kare (Chi-Square) Bağımsızlık Testi
# Bu test, inanç türü ile pratik arasındaki ilişkinin şans eseri olup olmadığını ölçer.
print("--- Ki-Kare Testi Sonuçları (P-Değerleri) ---")
print("(p < 0.05 ise istatistiksel olarak anlamlı bir ilişki var demektir)\n")

for practice in practices:
    # Çapraz tablo oluştur (Hangi inanç grubu, kaç kişi evet/hayır demiş)
    contingency_table = pd.crosstab(df['inanc_kategori'], df[practice])
    
    # Testi uygula
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    
    # Sonucu yazdır
    significance = "ANLAMLI FARK VAR" if p < 0.05 else "Anlamlı fark yok"
    print(f"{practice:15} -> p-value: {p:.5f}  ({significance})")

# 5. Görselleştirme: Isı Haritası (Heatmap)
plt.figure(figsize=(12, 7))

# Heatmap ayarları: annot=True sayıları gösterir, cmap renk paletidir.
sns.heatmap(practice_percentages, annot=True, fmt=".1f", cmap="YlGnBu", linewidths=.5)

plt.title('İnanç Gruplarına Göre Metafiziksel Pratiklerin Yapılma Oranı (%)', fontsize=14)
plt.ylabel('Kendini Tanımladığı İnanç', fontsize=12)
plt.xlabel('Metafiziksel Pratikler', fontsize=12)
plt.xticks(rotation=45) # Alt yazıların okunması için eğik yaz
plt.tight_layout()

# Grafiği kaydetmek istersen:
plt.savefig('inanc_pratik_analizi.png', dpi=300)

plt.show()